 # Práctica 3 - Computación Cuántica 2022-1
 ## John Camilo Giraldo López - Maury Marmolejo Din
 ### Para las especies asignadas: Alpha_A Paloma - Alpha_A Pato - Alpha_A Gallina
 ### Encontrar el porcentaje de similitud entre las especies, partiendo de su secuencia de nucleotidos codificandolos en circuitos cuánticos

In [1]:
Alpha_A_pigeon = "ATGGTGCTGTCTGCCAACGACAAGAGCAACGTGAAGGCCGTCTTCGGCAAAATCGGCGGC\
CAGGCCGGTGACTTGGGTGGTGAAGCCCTGGAGAGGTTGTTCATCACCTACCCCCAGACC\
AAGACCTACTTCCCCCACTTCGACCTGTCACATGGCTCCGCTCAGATCAAGGGGCACGGC\
AAGAAGGTGGCGGAGGCACTGGTTGAGGCTGCCAACCACATCGATGACATCGCTGGTGCC\
CTCTCCAAGCTGAGCGACCTCCACGCCCAAAAGCTCCGTGTGGACCCCGTCAACTTCAAA\
CTGCTGGGTCACTGCTTCCTGGTGGTCGTGGCCGTCCACTTCCCCTCTCTCCTGACCCCG\
GAGGTCCATGCTTCCCTGGACAAGTTCGTGTGTGCCGTGGGCACCGTCCTTACTGCCAAG\
TACCGTTAA"

Alpha_A_duck = "ATGGTGCTGTCTGCGGCTGACAAGACCAACGTCAAGGGTGTCTTCTCCAAAATCGGTGGC\
CATGCTGAGGAGTATGGCGCCGAGACCCTGGAGAGGATGTTCATCGCCTACCCCCAGACC\
AAGACCTACTTCCCCCACTTTGACCTGCAGCACGGCTCTGCTCAGATCAAGGCCCATGGC\
AAGAAGGTGGCGGCTGCCCTAGTTGAAGCTGTCAACCACATCGATGACATTGCGGGTGCT\
CTCTCCAAGCTCAGTGACCTCCACGCCCAAAAGCTCCGTGTGGACCCTGTCAACTTCAAA\
TTCCTGGGCCACTGCTTCCTGGTGGTGGTTGCCATCCACCACCCCGCTGCCCTGACCCCA\
GAGGTCCACGCTTCCCTGGACAAGTTCATGTGCGCCGTGGGTGCTGTGCTGACTGCCAAG\
TACCGTTAG"

Alpha_A_Chicken = "ATGGTGCTGTCCGCTGCTGACAAGAACAACGTCAAGGGCATCTTCACCAAAATCGCCGGC\
CATGCTGAGGAGTATGGCGCCGAGACCCTGGAAAGGATGTTCACCACCTACCCCCCAACC\
AAGACCTACTTCCCCCACTTCGATCTGTCACACGGCTCCGCTCAGATCAAGGGGCACGGC\
AAGAAGGTAGTGGCTGCCTTGATCGAGGCTGCCAACCACATTGATGACATCGCCGGCACC\
CTCTCCAAGCTCAGCGACCTCCATGCCCACAAGCTCCGCGTGGACCCTGTCAACTTCAAA\
CTCCTGGGCCAATGCTTCCTGGTGGTGGTGGCCATCCACCACCCTGCTGCCCTGACCCCG\
GAGGTCCATGCTTCCCTGGACAAGTTCTTGTGCGCCGTGGGCACTGTGCTGACCGCCAAG\
TACCGTTAA"


In [2]:
#Importación de librerías y clases para la simulación

import sys
import numpy as np
import math

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, BasicAer, IBMQ
from qiskit.visualization import plot_histogram, plot_bloch_multivector

from qiskit.extensions import Initialize

In [3]:
#Función para la codificación de los strings
def encode_bitstring(bitstring, qr, cr, inverse=False):
    """
    create a circuit for constructing the quantum superposition of the bitstring
    """
    n = math.ceil(math.log2(len(bitstring))) + 1                 #number of qubits
    assert n > 2, "the length of bitstring must be at least 2"
    
    qc = QuantumCircuit(qr, cr)
    
    #the probability amplitude of the desired state
    desired_vector = np.array([ 0.0 for i in range(2**n) ])     #initialize to zero

################################################################################################################    
 #         Cambio requerido en el código para corregir error presentado   #
    
    qc_init = QuantumCircuit(n) # Creación de compuertas circuitales para inicialización
    inverse_qc_init = QuantumCircuit(n) # Creación de compuertas circuitales para inversión de la inicialización

################################################################################################################

    amplitude = np.sqrt(1.0/2**(n-1))
    
    for i, b in enumerate(bitstring):
        pos = i * 2
        if b == "1" or b == "M":
            pos += 1
        desired_vector[pos] = amplitude
################################################################################################################    
 #         Cambio requerido en el código para corregir error presentado   #
    
    qc_init = Initialize(desired_vector)
    
 ################################################################################################################       
    
    if not inverse:
#         qc.initialize(desired_vector, qr)

        qc.append(qc_init, qr)  #  Cambio requerido en el código para corregir error presentado   #
    
        qc.barrier(qr)
        
    else:
#         qc.initialize(desired_vector, qr).inverse()  #invert the circuit

        inverse_qc_init = qc_init.gates_to_uncompute()  # Cambio requerido en el código para corregir error presentado   #
        qc.append(inverse_qc_init, qr)  # Cambio requerido en el código para corregir error presentado   #
        
        qc.barrier(qr)
        for i in range(n):
            qc.measure(qr[i], cr[i])
    return qc

In [4]:
#Función para la creación de los arreglos utilizados para realizar la comparación entre las cadenas de ADN
def circuitos(nombre_string,inverse=False):
    string=''
    array=[]
    array_a=[]# Arreglo de codificación de secuencias Adenina
    array_g=[]# Arreglo de codificación de secuencias Guanina
    array_t=[]# Arreglo de codificación de secuencias Timina
    array_c=[]# Arreglo de codificación de secuencias Citosina
    ban=0

    for i in (nombre_string):
        if ban<64:
            string = string + i #agregando datos al bloque
            ban+=1
        else:
            ban=0
            array.append(string) # Las secuencias se codifican en bloques de 64 caracteres y se guardan en un arreglo de bloques
            string=''

    while len(string)<64:
        string = string + '0' # Para la ultima cadena, se generan secuencias vacias para completar el bloque 
    array.append(string)

    for i in array: # Cada bloque se codifica en un nuevo arreglo
        for j in range(len(i)): # Cada caracter genera una columna
            if i[j] == 'A': # |00>
                array_a.append(1)
                array_g.append(0)
                array_t.append(0)
                array_c.append(0)
            elif i[j] == 'T': #|10>
                array_a.append(0)
                array_g.append(0)
                array_t.append(1)
                array_c.append(0)
            elif i[j] == 'G': #|01>
                array_a.append(0)
                array_g.append(1)
                array_t.append(0)
                array_c.append(0)
            elif i[j] == 'C': #|11>
                array_a.append(0)
                array_g.append(0)
                array_t.append(0)
                array_c.append(1)
            else:  # Los caracteres vacios generan columnas vacias, y sin significado cuantico
                array_a.append(0)
                array_g.append(0)
                array_t.append(0)
                array_c.append(0)

    n = math.ceil(math.log2(len(array_a[0:64]))) + 1 # Segeneran los n qubits partiendo del tamaño del bloque + 1 qubit que será llenado de acuerdo a la presencia del nucleotido                 
    qr = QuantumRegister(n)
    cr = ClassicalRegister(n)

    circuit_array_a=[]
    circuit_array_c=[]
    circuit_array_g=[]
    circuit_array_t=[]
    
    if not inverse:
        for k in range(0,len(array_a),64): # Se codifican los bloques y se agregan al arreglo de bloques de inicializacion del circuito cuantico respectivo
            circuit_array_a.append(encode_bitstring("".join([str(_) for _ in array_a[k:k+64]]), qr, cr))
            circuit_array_c.append(encode_bitstring("".join([str(_) for _ in array_c[k:k+64]]), qr, cr))
            circuit_array_g.append(encode_bitstring("".join([str(_) for _ in array_g[k:k+64]]), qr, cr))
            circuit_array_t.append(encode_bitstring("".join([str(_) for _ in array_t[k:k+64]]), qr, cr))
        
    else:
        for k in range(0,len(array_a),64):
            circuit_array_a.append(encode_bitstring("".join([str(_) for _ in array_a[k:k+64]]), qr, cr,inverse=True))
            circuit_array_c.append(encode_bitstring("".join([str(_) for _ in array_c[k:k+64]]), qr, cr,inverse=True))
            circuit_array_g.append(encode_bitstring("".join([str(_) for _ in array_g[k:k+64]]), qr, cr,inverse=True))
            circuit_array_t.append(encode_bitstring("".join([str(_) for _ in array_t[k:k+64]]), qr, cr,inverse=True))
    return [circuit_array_a,circuit_array_c,circuit_array_g,circuit_array_t] # Se retorna el arreglo que tiene todos los circuitos de nucleotidos, todos los bloques
#Estructura de la salida CircuitosEspecie{ nucleotido:{bloques};nucleotido:{bloques};nucleotido:{bloques};nucleotido:{bloques}}
#LLamado-> Especie[nucleotido][bloque]

In [5]:
# Creación de circuitos cuanticos    
array_Alpha_A_pigeon = circuitos(Alpha_A_pigeon)
array_Alpha_A_duck = circuitos(Alpha_A_duck)
array_Alpha_A_Chicken = circuitos(Alpha_A_Chicken)    

In [6]:
#Creación de los circuitos inversos para realizar la comparación
inverse_array_Alpha_A_pigeon = circuitos(Alpha_A_pigeon, inverse=True)
inverse_array_Alpha_A_duck = circuitos(Alpha_A_duck, inverse=True)
inverse_array_Alpha_A_Chicken = circuitos(Alpha_A_Chicken, inverse=True)

In [18]:
from qiskit import IBMQ, BasicAer

# Simulador local
n = 7 
backend = BasicAer.get_backend("qasm_simulator")
shots = 8192

combined_circs = {}
count = {}

most_similar, most_similar_score = "", -1.0 #Arreglos totales
sim_score1t = []
sim_score2t = []
sim_score3t = []

for i in range(4): # Para los 4 nucleotidos detectados compare la puntuación de cada nucleotido
    sim_score1 = []
    sim_score2 = []
    sim_score3 = []
    
    for j in range(7):# Todos los bloques
        combined_circs["pigeon_duck"] = array_Alpha_A_pigeon[i][j].compose(inverse_array_Alpha_A_duck[i][j])   #combined circuits to look for similar codes
        combined_circs["pigeon_Chicken"] = array_Alpha_A_pigeon[i][j].compose(inverse_array_Alpha_A_Chicken[i][j])
        combined_circs["Chicken_duck"] = array_Alpha_A_Chicken[i][j].compose(inverse_array_Alpha_A_duck[i][j])
        #print(combined_circs)
        job1 = execute(combined_circs["pigeon_duck"], backend=backend,shots=shots)
        job2 = execute(combined_circs["pigeon_Chicken"], backend=backend,shots=shots)
        job3 = execute(combined_circs["Chicken_duck"], backend=backend,shots=shots)
        st1 = job1.result().get_counts(combined_circs["pigeon_duck"])
        st2 = job2.result().get_counts(combined_circs["pigeon_Chicken"])
        st3 = job3.result().get_counts(combined_circs["Chicken_duck"])
        if "0"*n in st1:
            sim_score1.append(st1["0"*n]/shots)
        else:
            sim_score1.append(0.0)
        if "0"*n in st2:
            sim_score2.append(st2["0"*n]/shots)
        else:
            sim_score2.append(0.0)
        if "0"*n in st3:
            sim_score3.append(st3["0"*n]/shots)
        else:
            sim_score3.append(0.0)
    plot_histogram([st1,st2,st3],legend=['Paloma_Pato', 'Paloma_Gallina','Gallina_Pato'])
    sim_score1t.append(np.mean(sim_score1))
    sim_score2t.append(np.mean(sim_score2))
    sim_score3t.append(np.mean(sim_score3))
print('Similitud entre paloma y pato: ',np.mean(sim_score1t))
print('Similitud entre paloma y gallina: ',np.mean(sim_score2t))
print('Similitud entre gallina y pato: ',np.mean(sim_score3t))

Similitud entre paloma y pato:  0.8595450265066964
Similitud entre paloma y gallina:  0.8637346540178572
Similitud entre gallina y pato:  0.89093017578125


## Prueba de buen funcionamiento

In [11]:
Alpha_A_pigeon = "ATGGTGCTGTCTGCCAACGACAAGAGCAACGTGAAGGCCGTCTTCGGCAAAATCGGCGGC\
CAGGCCGGTGACTTGGGTGGTGAAGCCCTGGAGAGGTTGTTCATCACCTACCCCCAGACC\
AAGACCTACTTCCCCCACTTCGACCTGTCACATGGCTCCGCTCAGATCAAGGGGCACGGC\
AAGAAGGTGGCGGAGGCACTGGTTGAGGCTGCCAACCACATCGATGACATCGCTGGTGCC\
CTCTCCAAGCTGAGCGACCTCCACGCCCAAAAGCTCCGTGTGGACCCCGTCAACTTCAAA\
CTGCTGGGTCACTGCTTCCTGGTGGTCGTGGCCGTCCACTTCCCCTCTCTCCTGACCCCG\
GAGGTCCATGCTTCCCTGGACAAGTTCGTGTGTGCCGTGGGCACCGTCCTTACTGCCAAG\
TACCGTTAA"
Alpha_A_pigeon_mod = "ATGGTGCTGTCTGCCAACGACAAGAGCAACGTGAAGGCCGTCTTCGGCAAAATCGGCGGC\
CAGGCCGGTGACTTGGGTGGTGAAGCCCTGGAGAGGTTGTTCATCACCTACCCCCAGACC\
AAGACCTACTTCCCCCACTTCGACCTGTCACATGGCTCCGCTCAGATCAAGGGGCACGGC\
AAGAAGGTGGCGGAGGCACTGGTTGAGGCTGCCAACCACATCGATGACATCGCTGGTGCC\
CTCTCCAAGCTGAGCGACCTCCACGCCCAAAAGCTCCGTGTGGACCCCGTCAACTTCAAA\
CTGCTGGGTCACTGCTTCCTGGTGGTCGTGGCCGTCCACTTCCCCTCTCTCCTGACCCCG\
GAGGTCCATGCTTCCCTGGACAAGTTCGTGTGTGCCGTGGGCACCGTCCTTACTGCCAAT\
TACCGTTAA"

In [12]:
array_Alpha_A_pigeon = circuitos(Alpha_A_pigeon)
array_Alpha_A_pigeon_mod = circuitos(Alpha_A_pigeon_mod)

In [13]:
inverse_array_Alpha_A_pigeon = circuitos(Alpha_A_pigeon, inverse=True)
inverse_array_Alpha_A_pigeon_mod = circuitos(Alpha_A_pigeon_mod, inverse=True)

In [15]:
from qiskit import IBMQ, BasicAer

# Simulador local
n = 7
backend = BasicAer.get_backend("qasm_simulator")
shots = 8192

combined_circs = {}

sim_score1t = []
sim_score2t = []

for i in range(4): # Para los 4 nucleotidos detectados compare la puntuación de cada nucleotido
    sim_score1 = []
    sim_score2 = []
    for j in range(7):# Todos los bloques
        combined_circs["pigeon_pigeon"] = array_Alpha_A_pigeon[i][j].compose(inverse_array_Alpha_A_pigeon[i][j])   #combined circuits to look for similar codes
        combined_circs["pigeon_pigeon_mod"] = array_Alpha_A_pigeon[i][j].compose(inverse_array_Alpha_A_pigeon_mod[i][j])   #combined circuits to look for similar codes
        #print(combined_circs)
        job1 = execute(combined_circs["pigeon_pigeon"], backend=backend,shots=shots)
        job2 = execute(combined_circs["pigeon_pigeon_mod"], backend=backend,shots=shots)
        st1 = job1.result().get_counts(combined_circs["pigeon_pigeon"])
        st1 = job2.result().get_counts(combined_circs["pigeon_pigeon_mod"])
        #print(st)
        if "0"*n in st1:
            sim_score1.append(st1["0"*n]/shots)
        else:
            sim_score1.append(0.0)
        if "0"*n in st2:
            sim_score2.append(st2["0"*n]/shots)
        else:
            sim_score2.append(0.0)
    sim_score1t.append(np.mean(sim_score1))
    sim_score2t.append(np.mean(sim_score2))
print('Similitud de paloma consigomisma: ',np.mean(sim_score1t))
print('Similitud de paloma con su modificación: ',np.mean(sim_score2t))

Similitud de paloma consigomisma:  0.9978550502232143
Similitud de paloma con su modificación:  0.8782958984375


In [21]:
from qiskit.providers.ibmq import least_busy
from qiskit.tools.monitor import job_monitor
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-education', group='uni-antioquia-1', project='qcudea2022-1')
remote_backend = least_busy(provider.backends(filters=lambda x: x.configuration().n_qubits >=7
                                           and not x.configuration().simulator
                                           and x.status().operational==True))
print("El computador cuantico de 7 bits menos ocupado es el: "+remote_backend.name())
backend = provider.get_backend(remote_backend.name())

ibmqfactory.load_account:WARNING:2022-08-06 18:41:45,444: Credentials are already in use. The existing account in the session will be replaced.


El computador cuantico de 7 bits menos ocupado es el: ibm_perth


In [22]:
from qiskit import IBMQ, BasicAer

# Simulador local
n = 7
#backend = BasicAer.get_backend("qasm_simulator")
shots = 1024

combined_circs = {}

sim_score1t = []
sim_score2t = []

for i in range(4): # Para los 4 nucleotidos detectados compare la puntuación de cada nucleotido
    sim_score1 = []
    sim_score2 = []
    for j in range(7):# Todos los bloques
        combined_circs["pigeon_pigeon"] = array_Alpha_A_pigeon[i][j].compose(inverse_array_Alpha_A_pigeon[i][j])   #combined circuits to look for similar codes
        combined_circs["pigeon_pigeon_mod"] = array_Alpha_A_pigeon[i][j].compose(inverse_array_Alpha_A_pigeon_mod[i][j])   #combined circuits to look for similar codes
        #print(combined_circs)
        job1 = execute(combined_circs["pigeon_pigeon"], backend=backend,shots=shots)
        job2 = execute(combined_circs["pigeon_pigeon_mod"], backend=backend,shots=shots)
        st1 = job1.result().get_counts(combined_circs["pigeon_pigeon"])
        st1 = job2.result().get_counts(combined_circs["pigeon_pigeon_mod"])
        #print(st)
        if "0"*n in st1:
            sim_score1.append(st1["0"*n]/shots)
        else:
            sim_score1.append(0.0)
        if "0"*n in st2:
            sim_score2.append(st2["0"*n]/shots)
        else:
            sim_score2.append(0.0)
    sim_score1t.append(np.mean(sim_score1))
    sim_score2t.append(np.mean(sim_score2))
print('Similitud de paloma consigomisma: ',np.mean(sim_score1t))
print('Similitud de paloma con su modificación: ',np.mean(sim_score2t))

Similitud de paloma consigomisma:  0.013427734375
Similitud de paloma con su modificación:  7.021484375


In [8]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
